In [4]:
import requests
from bs4 import BeautifulSoup
import pymysql
import json
import time
import re

In [45]:
def get_post_data(strrec):
    data = json.loads(r'''{
        "find_key1": "關鍵字",
        "search_work": "軟體工程師",
        "find_work_mode1": "2_1011_0001_0005",
        "s_find_work_mode1": "軟體工程師",
        "strrec": %s,
        "search_type": "job",
        "search_item": "1",
        "search_from": "index"}'''%(strrec))
    return data

def get_headers():
    headers = json.loads(r'''{
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7",
        "Cache-Control": "max-age=0",
        "Connection": "keep-alive",
        "Content-Length": "4419",
        "Content-Type": "application/x-www-form-urlencoded",
        "Cookie": "_sh_c_1=search_type%3Ajob%7Csearch_item%3A1%7Cs_find_work_mode1%3A%E8%BB%9F%E9%AB%94%E5%B7%A5%E7%A8%8B%E5%B8%AB%7Cfind_work_mode1%3A2_1011_0001_0005%7C_mu_ckb_21%3Acheckbox%7C_mu_ckb_22%3Acheckbox%7C_mu_ckb_23%3Acheckbox%7C_mu_ckb_24%3Acheckbox%7C_mu_ckb_25%3Acheckbox; _sh_c_2=search_type%3Ajob%7Csearch_item%3A1%7Cs_find_work_mode1%3A%E8%BB%9F%E9%AB%94%E5%B7%A5%E7%A8%8B%E5%B8%AB%7Cfind_work_mode1%3A2_1011_0001_0005%7C_mu_ckb_21%3Acheckbox%7C_mu_ckb_22%3Acheckbox%7C_mu_ckb_23%3Acheckbox%7C_mu_ckb_24%3Acheckbox%7C_mu_ckb_25%3Acheckbox; _ga=GA1.3.1697330314.1516179527; ASPSESSIONIDSSDBATTA=LMBMJICCHJBAPBOEGMGKJNPF; ASPSESSIONIDSSARRSRQ=ABIHPEBCNGMJIANDINJBOIOI; _gid=GA1.3.509086006.1516527407; yes123_make_cookie=cc1a299f7899bf32341f07b7ab716f16; view%5Fjob%5Fid%5Fstr=20171030160315%5F23750095%40%21%4020171110154748%5F5061151%40%21%40%E7%A8%8B%E5%BC%8F%E7%B3%BB%E7%B5%B1%E5%B7%A5%E7%A8%8B%E5%B8%AB%24%21%2420171121100345%5F53216057%40%21%4020180108084033%5F24394301%40%21%40%E8%BB%9F%E9%AB%94%E7%B6%AD%E8%AD%B7%E5%B7%A5%E7%A8%8B%E5%B8%AB; ASPSESSIONIDQSDABSSA=OJLIBGCCDFENIJOKNNCOLDFC; ASPSESSIONIDQQCBASTA=HMOHDCECDEJLOHHIAGHMHOAJ; ASPSESSIONIDSSCTTSSR=ACOFJBNBKCPJHOGMGCJAHJNF; ASPSESSIONIDQQADBTSA=AAFOLEECEACBLFJEMGPEJIDC; citrix_ns_id=BzZl3GUO9H+Fqy0PdyubyQ2TckwA000; ASP.NET_SessionId=608953571; StepCookie_id=608953571; ClientIP=122.116.5.248; _gat_UA-4824195-2=1; step=45",
        "Host": "www.yes123.com.tw",
        "Origin": "https://www.yes123.com.tw",
        "Referer": "https://www.yes123.com.tw/admin/index.asp",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36"
    }''')
    return headers
    
def get_url_soup(url, strrec):
    resp = requests.post(url, data=get_post_data(strrec), headers=get_headers())
    resp.encoding = 'utf-8'
    soup = BeautifulSoup(resp.text, 'html5lib')
    return soup

def get_url_list(url, strrec):
    soup = get_url_soup(url, strrec)
    url_list = []
    for jobname in [a for a in soup.select('a.jobname') if a.has_attr('href')]:
        url_list.append(home + jobname['href'])
    return url_list

def get_detail(url):
    soup = get_url_soup(url, 0)
    
    # 初始Job化欄位
    job = {}
    job['title'] = ''
    job['company'] = ''
    job['location'] = ''
    job['skills'] = ''
    
    # 職位
    span = soup.select_one('.jobname_title > span')
    if span is not None:
        job['title'] = span.previousSibling
    else:
        return job
    
    # 公司名稱
    span = soup.select_one('div.jobname_title p a')
    if span is not None:
        job['company'] = span.text
    
    # 工作地點
    comp_titles = soup.select('.comp_detail h2')
    hire_title = [title for title in comp_titles if title.text == '徵才說明']
    job['location'] = hire_title[0].findNext('ul').select('span')[3].text.strip().split()[0]
    
    # 工作技能
    skill_title = [title for title in comp_titles if title.text == '技能與求職專長']
    skill_spans = skill_title[0].findNext('ul').select('span.rr')
    for span in skill_spans:
        skills_list = span.text.split('：')
        if len(skills_list) > 1: 
            job['skills'] += skills_list[1].replace('Java Script', 'JavaScript').replace('／', ' ').replace('/', ' ') + ' '
    return job

def getTotalPage(url):
    soup = get_url_soup(url, 0)
    td_text = soup.select('table.sift td')[2].text.replace(' ','')
    total_page = re.search(r'/(\d+)頁', td_text).group(1)
    return total_page

In [46]:
home = 'https://www.yes123.com.tw/admin/'
search_url = "https://www.yes123.com.tw/admin/job_refer_list.asp"
total_page = int(getTotalPage(search_url)) + 1

conn = pymysql.connect(host='localhost', port=3306, user='root', password='1234', db='jobs', use_unicode=True, charset="utf8mb4", autocommit=True)
cur = conn.cursor()
insert_1111_template = "insert into job1(site, company, title, location, skills) values('yes123', %s, %s, %s, %s)"
print('Total pages: ' + str(total_page))

for page1 in range(0, total_page):
    time.sleep(1)
    print('page: ' + str(page1+1))
    page2 = page1 * 20
    url_list = get_url_list(search_url, page2) # 查詢頁抓取每筆工作連結

    job_list = []
    for job_url in url_list:
        job = get_detail(job_url) # 內容頁抓取工作資訊
        job_list.append(job)
        
        jobs = [(job['company'], job['title'], job['location'], job['skills']) for job in job_list if job['company'] != '']
        cur.executemany(insert_1111_template, jobs)

cur.close
conn.close

Total pages: 27
page: 1
page: 2
page: 3
page: 4
page: 5
page: 6
page: 7
page: 8
page: 9
page: 10
page: 11
page: 12
page: 13
page: 14
page: 15
page: 16
page: 17
page: 18
page: 19
page: 20
page: 21
page: 22
page: 23
page: 24
page: 25
page: 26
page: 27


<bound method Connection.close of <pymysql.connections.Connection object at 0x09C6B270>>